In [27]:
class CaseInsensitively(object):
    def __init__(self, s):
        self.__s = s.lower()
    def __hash__(self):
        return hash(self.__s)
    def __eq__(self, other):
        # ensure proper comparison between instances of this class
        try:
           other = other.__s
        except (TypeError, AttributeError):
          try:
             other = other.lower()
          except:
             pass
        return self.__s == other

In [28]:
def get_eventbrite_list(list_type='contact'):
    return get_contact_list() if list_type == 'contact' else create_event_name_column(get_event_list())


def get_contact_list():
    number_of_pages = connect().get_user_owned_event_orders(user_id())['pagination']['page_count']
    
    return [
        connect().get_user_owned_event_orders(user_id(), page=page_number)
        for page_number in tqdm(range(1, number_of_pages + 1), total=number_of_pages, desc="Contact Pages to Scrape")
    ]


def create_event_name_column(event_list):
    for i in range(0, len(event_list)):
        for event in range(0, number_of_events):
            event_list[i]['events'][event]['event_name'] = event_list[i]['events'][event]['name']['text']
    
    return event_list


def get_event_list():
    number_of_pages = connect().get_user_owned_events(user_id())['pagination']['page_count']
    
    return [
        connect().get_user_owned_events(user_id(), page=page_number)
        for page_number in tqdm(range(1, number_of_pages + 1), total=number_of_pages, desc="Event Pages to Scrape")
    ]

In [21]:
def convert_eventbrite_list_to_dataframe(eventbrite_list, list_type='contact'):
    df = pd.DataFrame()
    
    return convert_contact_list_to_dataframe(eventbrite_list, df) if list_type == 'contact'\
           else convert_event_list_to_dataframe(eventbrite_list, df)


def convert_contact_list_to_dataframe(contact_list, df):
    for i in range(0, len(contact_list)):
        df = df.append(contact_list[i]['orders'])
    return df.rename(columns={'name': 'full_name', 'id': 'contact_id'})
    
    
def convert_event_list_to_dataframe(event_list, df):
    for i in range(0, len(event_list)):
        df = df.append(event_list[i]['events'])
    return df.rename(columns={'id': 'event_id'})

In [20]:
def combined_contacts_and_events(contacts_dataframe, events_dataframe):
    return pd.concat(
        [
            contacts_dataframe.set_index('event_id'),
            events_dataframe.set_index('event_id')
        ], axis=1, join='inner'
    ).reset_index()

In [22]:
def create_event_name_column(event_list):
    for i in range(0, len(event_list)):
        for event in range(0, number_of_events):
            event_list[i]['events'][event]['event_name'] = event_list[i]['events'][event]['name']['text']
    
    return event_list